In [5]:
import pandas as pd
import os

def analyser_structure_fichier():
    print("=== ANALYSE DE LA STRUCTURE DU FICHIER ===")
    
    chemin_fichier = 'hackaton_projet_W3D5/DATASET/plats_populaires_ingredients.csv'
    
    try:
        # Lecture du fichier avec différents séparateurs
        print("\nTentative de lecture avec différents séparateurs...")
        
        # Essai avec point-virgule
        df_semicolon = pd.read_csv(chemin_fichier, sep=';', nrows=5)
        print("\nLecture avec point-virgule :")
        print("Colonnes trouvées :", df_semicolon.columns.tolist())
        print("\nPremières lignes :")
        print(df_semicolon.head(2))
        
        # Essai avec virgule
        df_comma = pd.read_csv(chemin_fichier, sep=',', nrows=5)
        print("\nLecture avec virgule :")
        print("Colonnes trouvées :", df_comma.columns.tolist())
        print("\nPremières lignes :")
        print(df_comma.head(2))
        
        # Affichage des informations détaillées
        print("\nInformations détaillées du DataFrame :")
        print(df_semicolon.info())
        
        # Vérification des valeurs nulles
        print("\nVérification des valeurs manquantes :")
        print(df_semicolon.isnull().sum())
        
    except Exception as e:
        print(f"Erreur lors de l'analyse : {e}")

# Exécution
if __name__ == "__main__":
    analyser_structure_fichier()


=== ANALYSE DE LA STRUCTURE DU FICHIER ===

Tentative de lecture avec différents séparateurs...

Lecture avec point-virgule :
Colonnes trouvées : ['Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients']

Premières lignes :
  Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients
0  Beef Brisket Pot Roast,ÉTATS-UNIS,Viande,Beef ...
1  Big Mac,ÉTATS-UNIS,Fast-food,Minced Beef (400g...

Lecture avec virgule :
Colonnes trouvées : ['Plat', 'Pays', 'Catégorie', 'Ingrédients', 'Nombre_Ingrédients']

Premières lignes :
                     Plat        Pays  Catégorie  \
0  Beef Brisket Pot Roast  ÉTATS-UNIS     Viande   
1                 Big Mac  ÉTATS-UNIS  Fast-food   

                                         Ingrédients  Nombre_Ingrédients  
0  Beef Brisket (4-5 pound) | Salt (Dash) | Onion...                  11  
1  Minced Beef (400g) | Olive Oil (2 tbs) | Sesam...                  14  

Informations détaillées du DataFrame :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4

In [11]:
import os
import pandas as pd
import requests
import time
from typing import Dict, List

class NutrientAnalyzer:
    def __init__(self):
        self.API_KEY = "V47bTvrwdE63sv3FPzRPbyo30fxLmg4cJnc6tWol"
        self.base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
        self.nutriments_cibles = {
            'Protein': 'Protéines (g)',
            'Total lipid (fat)': 'Lipides (g)',
            'Carbohydrate, by difference': 'Glucides (g)',
            'Energy': 'Calories (kcal)',
            'Fiber, total dietary': 'Fibres (g)',
            'Sugars, total including NLEA': 'Sucres (g)'
        }
        
        self.chemin_base = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET"
        self.chemin_fichier = os.path.join(self.chemin_base, "plats_populaires_ingredients.csv")
        self.chemin_sortie = os.path.join(self.chemin_base, "plats_populaires_nutriments_complet.csv")

    def extraire_ingredients(self, ingredients_str: str) -> List[str]:
        if pd.isna(ingredients_str) or ingredients_str == 'Non disponible':
            return []
        return [ing.split('(')[0].strip() for ing in ingredients_str.split('|')]

    def rechercher_nutriments(self, ingredient: str) -> Dict:
        params = {
            'api_key': self.API_KEY,
            'query': ingredient,
            'pageSize': 1,
            'dataType': ['Foundation', 'Survey (FNDDS)']
        }
        
        try:
            response = requests.get(self.base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                if data.get('foods'):
                    nutrients = data['foods'][0]['foodNutrients']
                    return {n['nutrientName']: n['value'] for n in nutrients}
            return {}
        except Exception as e:
            print(f"Erreur pour {ingredient}: {e}")
            return {}

    def analyser_plats(self):
        try:
            print(f"Lecture du fichier : {self.chemin_fichier}")
            # Changement ici : lecture avec virgule comme séparateur
            df = pd.read_csv(self.chemin_fichier, sep=',')
            print(f"Dataset chargé avec succès : {len(df)} plats")
            
            # Affichage des colonnes pour débogage
            print("\nColonnes disponibles :", df.columns.tolist())
            
            resultats = []
            total_plats = len(df)
            
            for index, row in df.iterrows():
                plat = row['Plat'].strip()  # Ajout de strip() pour nettoyer les espaces
                print(f"\nAnalyse du plat {index + 1}/{total_plats} : {plat}")
                
                ingredients = self.extraire_ingredients(row['Ingrédients'])
                nutriments_totaux = {value: 0 for value in self.nutriments_cibles.values()}
                
                for ingredient in ingredients:
                    print(f"  Analyse de : {ingredient}")
                    nutriments = self.rechercher_nutriments(ingredient)
                    
                    for api_name, fr_name in self.nutriments_cibles.items():
                        if api_name in nutriments:
                            nutriments_totaux[fr_name] += nutriments[api_name]
                    
                    time.sleep(1)
                
                resultat = {
                    'Plat': plat,
                    'Pays': row['Pays'].strip(),
                    'Catégorie': row['Catégorie'].strip()
                }
                resultat.update(nutriments_totaux)
                
                resultats.append(resultat)
                
                if (index + 1) % 5 == 0:
                    df_temp = pd.DataFrame(resultats)
                    temp_path = os.path.join(self.chemin_base, "plats_populaires_nutriments_temp.csv")
                    df_temp.to_csv(temp_path, index=False, encoding='utf-8-sig', sep=',')

            df_nutriments = pd.DataFrame(resultats)
            df_nutriments.to_csv(self.chemin_sortie, index=False, encoding='utf-8-sig', sep=',')
            
            print("\nMoyennes nutritionnelles par pays :")
            colonnes_nutriments = list(self.nutriments_cibles.values())
            stats_pays = df_nutriments.groupby('Pays')[colonnes_nutriments].mean()
            print(stats_pays.round(2))
            
            return df_nutriments

        except Exception as e:
            print(f"Erreur lors de l'analyse : {e}")
            import traceback
            print(traceback.format_exc())
            return None

# Exécution
if __name__ == "__main__":
    print("=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===")
    print(f"Dossier de travail actuel : {os.getcwd()}")
    analyzer = NutrientAnalyzer()
    df_resultat = analyzer.analyser_plats()




=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===
Dossier de travail actuel : /Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5
Lecture du fichier : /Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_ingredients.csv
Dataset chargé avec succès : 42 plats

Colonnes disponibles : ['Plat', 'Pays', 'Catégorie', 'Ingrédients', 'Nombre_Ingrédients']

Analyse du plat 1/42 : Beef Brisket Pot Roast
  Analyse de : Beef Brisket
  Analyse de : Salt
  Analyse de : Onion
  Analyse de : Garlic
  Analyse de : Thyme
  Analyse de : Rosemary
  Analyse de : Bay Leaves
  Analyse de : beef stock
  Analyse de : Carrots
  Analyse de : Mustard
  Analyse de : Potatoes

Analyse du plat 2/42 : Big Mac
  Analyse de : Minced Beef
  Analyse de : Olive Oil
  Analyse de : Sesame Seed Burger Buns
  Analyse de : Onion
  Analyse de : Iceberg Lettuce
  Analyse de : Cheese
  Analyse de : Dill Pickles
  Analyse de : Mayonnaise
  An